In [1]:
import numpy as np
import random
import html
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml
import country_converter
import rapidfuzz
import tqdm
import matplotlib as plt
import time
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import glob
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
df_dahua = pd.read_csv(r'C:\Users\LENOVO\Documents\GitHub\Enrichment_database\data\dahua_df.csv')
df_dahua.head()

,Unnamed: 0,year,name,name_clean,name_canonical,description,slug,website,email,linkedin,twitter,instagram,facebook,employee_count,size,industry,type,country,founded,locality,latest_funding_raw,investors,total_funding,number_of_funding_rounds,business_model_type,latest_funding_amount,latest_funding_currency,source
0,2,2025,DAHUA TECHNOLOGY,NaN,NaN,Dahua Technology is a world-leading video-cent...,NaN,https://www.dahuatech.com,du_rui1@dahuatech.com,https://www.linkedin.com/company/dahua-technology,NaN,https://www.instagram.com/dahuaglobal,NaN,0,NaN,ENABLING TECHNOLOGIES,NaN,CN,0,"HANGZHOU, ZHEJIANG",NaN,NaN,0.00,0,NaN,0.00,NaN,SMART_CITY


In [3]:
df_dahua_mock = pd.read_csv(r'C:\Users\LENOVO\Documents\GitHub\Enrichment_database\data\dahua_mockup_df.csv')
df_dahua_mock.head()

,Unnamed: 0,name,website,exhibitor_email,phone,city,zip_code,company_address,industry,description,contact_email,contact_name,contact_phone,event_sap_code,instagram,linkedin,stand_location,year,country,cb_rank,growth_score,heat_score,operating_status,company_type,founded_date,founders,num_funding_rounds,total_funding_amount,latest_funding_type,num_investors,lead_investors,num_lead_investors,num_acquisitions,acquisition_prediction,ipo_prediction,funding_prediction,prediction_last_updated,headquarters_city,headquarters_country,employee_count_range,stock_symbol,ipo_date,num_investments,latest_investment,top_contributing_factors,investor_table_data,grant_round_links,acquisitions_list,num_transactions_visible,logo_url,linkedin_about,linkedin_tagline,linkedin_followers,linkedin_employee_count,linkedin_headquarters,linkedin_founded_year,linkedin_specialties,linkedin_people_profiles,linkedin_jobs_count,linkedin_recent_post,linkedin_type,linkedin_industry,tech_stack,monthly_web_visits,web_traffic_growth_pct,it_spend_estimate,trademark_count,patent_count,domain_age_years,domain_hosting_country,num_orgs_in_hub,total_hub_funding,num_investors_in_hub,hub_category,trend_category,founding_amount,founding_last_round,last_funding_date,marketcap,company_value,Products
0,0,Dahua Technology,https://www.dahuatech.com,contact@dahuatech.com,+86-571-2899-8999,Hangzhou,310053,"No. 1199 Bin’an Road, Binjiang District, Hangz...","Security, Surveillance, AI",Dahua Technology provides video-centric smart ...,pr@dahuatech.com,Fu Liquan,+86-571-2899-8999,E-SMART-2025-DAHUA,https://www.instagram.com/dahua.tech,https://www.linkedin.com/company/dahua-technol...,"Hall 5, Booth 214",2025,China,18243,91,93,Active,For Profit,2001,Fu Liquan and Zhu Jiangming,2,$1.9B,Series C,3,"Sequoia Capital China, Hillhouse Capital",2,1,Probable,Unlikely,No future rounds expected,2025-10-18,Hangzhou,China,"10,001+",SZSE:002236,NaN,3,Renxin Technology,Partnership Announcement,"[{'Investor': 'Hillhouse Capital', 'Lead': Tru...",['https://www.crunchbase.com/funding_round/dah...,['Acquired XVR Systems (2022)'],5,https://logo.clearbit.com/dahuatech.com,Dahua Technology is a world-leading video-cent...,Enabling a safer society and smarter living.,237000,18500,"Hangzhou, Zhejiang, China",2001,"['AI', 'Video Surveillance', 'IoT', 'Security']","[{'Name': 'Fu Liquan', 'Role': 'Chairman'}, {'...",12,Dahua launches new full-color AI cameras at GI...,Public,Information Technology / Security / Video,"['Cloudflare', 'Google Analytics', 'AWS CloudF...",1.8M,3.2%,$320K,5,12,24,China,9887,$1.9T,51449,Financial Services Companies With More Than 10...,Among all Financial Services companies,TO BE ENRICHED,TO BE ENRICHED,TO BE ENRICHED,TO BE ENRICHED,TO BE ENRICHED,TO BE ENRICHED


In [4]:
df_dahua_extracted = pd.read_csv(r'C:\Users\LENOVO\Documents\GitHub\Enrichment_database\data\dahua-technology_overview_extracted.csv')
df_dahua_extracted.head()

,cb_rank,founded_date,legal_name,operating_status,stock_symbol,company_profit_type,founders,phone_number,contact_email,monthly_web_visits,web_traffic_growth,location_identifiers,location_city,location_region,location_country,description_short,company_type,last_funding_type,categories,last_funding_amount,last_funding_date,lead_investor,num_employees,activity_trend,key_people,about_company,investments_count,employee_profiles_count,investments_announced_date,investments_transaction_name,investments_number_of_investors,investments_money_raised,investments_lead_investors,investments_announced_dates,investments_announced_date1,investments_announced_date2,investments_announced_date3,investments_organization_names,investments_organization_name1,investments_organization_name2,investments_organization_name3,investments_money_raised1,investments_money_raised2,investments_money_raised3,news_dates,news_sources,news_source1,news_source2,news_source3,news_source4,news_headlines,news_headline1,news_headline2,news_headline3,news_headline4,news_links,news_link1,news_link2,news_link3,news_link4,number_of_funding_rounds,faqs,faqs_list,faqs1,faqs1_answer,faqs2,faqs2_answer,faqs3,faqs3_answer,faqs4,faqs4_answer,interest_signals_table,interest_signals_list,interest_signal1_topic,interest_signal1_score,interest_signal2_topic,interest_signal2_score,interest_signal3_topic,interest_signal3_score,actively_used_products_table,actively_used_products_amount,actively_used_products_list,actively_used_product1,actively_used_product2,actively_used_product3,active_website_tech_table,active_website_tech_count,active_website_tech_table_list,active_website_tech1,active_website_tech2,active_website_tech3,ip_table,ip_patents_count,ip_primar_patents_category,ip_trademarks_count,ip_trademarks_most_popular_class
0,24411,"Mar 12, 2001","Zhejiang Dahua Technology Co., Ltd",Active,SZSE:002236,For Profit,Fu Fiquan,8657187688883,overseas@dahuatech.com,455812,11.02,"Hangzhou, Zhejiang, China, Hangzhou, Zhejiang,...",Hangzhou,Zhejiang,China,Dahua Technology is a world-leading video surv...,For Profit,Post-IPO Equity,"Information Technology, Security, Video",CN¥5.1B,Mar 2021,China Mobile,10001+,Among all Video companies,"Zhang Xingming: Executive President, Wu Jian: ...",Dahua Technology is a world-leading video surv...,3,14,"Mar 29, 2021",Post-IPO Equity - Dahua Technology,1,"— , $150K , CN¥30M",China Mobile,"Apr 22, 2025, Feb 13, 2018, Aug 7, 2017","Apr 22, 2025","Feb 13, 2018","Aug 7, 2017","Renxin Technology, Adjoint, MAYCUR.COM",Renxin Technology,Adjoint,MAYCUR.COM,—,$150K,CN¥30M,NaN,"Boston Globe, Investing.com, Tech in Asia, Inv...",Boston Globe,Investing.com,Tech in Asia,Investing.com,US government allowed and even helped US firms...,US government allowed and even helped US firms...,US agency votes to tighten restrictions on Chi...,US agency tightens curbs on Chinese tech firms...,FCC moves to bar Hong Kong telecom carrier fro...,https://www.bostonglobe.com/2025/10/29/nation/...,https://www.bostonglobe.com/2025/10/29/nation/...,https://www.investing.com/news/stock-market-ne...,https://www.techinasia.com/news/agency-tighten...,https://www.investing.com/news/stock-market-ne...,1,Where is Dahua Technology's headquarters?Dahua...,"(""Where is Dahua Technology's headquarters?"", ...",Where is Dahua Technology's headquarters?,"Dahua Technology is located in Hangzhou, Zheji...",Who invested in Dahua Technology?,Dahua Technology is funded by China Mobile.,When was the last funding round for Dahua Tech...,Dahua Technology closed its last funding round...,Who are Dahua Technology's competitors?,Alternatives and possible competitors to Dahua...,| Topic | Surge Score | | --- | --- | | ![Comm...,"('Common Access Card (CAC)', '100'), ('Ericsso...",Common Access Card (CAC),100,Ericsson,99,Incident and Problem Management,98,4 Dahua Technology uses 4 technology products ...,4,"Microsoft Outlook, DigiCert, GlobalSign",Microsoft Outlook,DigiCert,GlobalSign,36 Dahua Technology is actively 

In [6]:
df_dahua_apollo = pd.read_csv(r'C:\Users\LENOVO\Documents\GitHub\Enrichment_database\data\Apollo\dataset_leads-finder_2025-12-10_16-53-58-541.csv')
df_dahua_apollo.head()

,city,company_annual_revenue,company_annual_revenue_clean,company_city,company_country,company_description,company_domain,company_founded_year,company_full_address,company_linkedin,company_linkedin_uid,company_name,company_phone,company_postal_code,company_size,company_state,company_street_address,company_technologies,company_total_funding,company_total_funding_clean,company_website,country,email,first_name,full_name,functional_level,headline,industry,job_title,keywords,last_name,linkedin,mobile_number,personal_email,seniority_level,state
0,NaN,32390000000.00,32.4B,Hangzhou Shi,China,"Zhejiang Dahua Technology Co., Ltd. (Dahua Tec...",dahuasecurity.com,2001,"No.1199, Bin'an Road, Binjiang District, Hangz...",http://www.linkedin.com/company/dahua-technology,2845844,Dahua Technology Co. Ltd,+86 571 8768 8883,NaN,23000,Zhejiang Sheng,1199 Bin'an Road,"hubspot, react redux, remote, route 53, vuejs",NaN,NaN,http://www.dahuasecurity.com,Croatia,damir.klarin@dahuatech.com,Damir,Damir Klarin,sales,NaN,Information Technology & Services,Business Development Manager,"dvrs, nvr, ivs, ip cameras, security solution ...",Klarin,http://www.linkedin.com/in/damir-klarin-8496283,NaN,NaN,manager,NaN
1,Dubai,32390000000.00,32.4B,Hangzhou Shi,China,"Zhejiang Dahua Technology Co., Ltd. (Dahua Tec...",dahuasecurity.com,2001,"No.1199, Bin'an Road, Binjiang District, Hangz...",http://www.linkedin.com/company/dahua-technology,2845844,Dahua Technology Co. Ltd,+86 571 8768 8883,NaN,23000,Zhejiang Sheng,1199 Bin'an Road,"hubspot, react redux, remote, route 53, vuejs",NaN,NaN,http://www.dahuasecurity.com,United Arab Emirates,NaN,Ali,Ali Kloub,sales,Regional Director (mena)\nbusiness Growth | St...,Information Technology & Services,Regional Sales Director (mena),"dvrs, nvr, ivs, ip cameras, security solution ...",Kloub,http://www.linkedin.com/in/ali-kloub-b15b2657,NaN,alikloub@hotmail.com,director,Dubai
2,Jeddah,32390000000.00,32.4B,Hangzhou Shi,China,"Zhejiang Dahua Technology Co., Ltd. (Dahua Tec...",dahuasecurity.com,2001,"No.1199, Bin'an Road, Binjiang District, Hangz...",http://www.linkedin.com/company/dahua-technology,2845844,Dahua Technology Co. Ltd,+86 571 8768 8883,NaN,23000,Zhejiang Sheng,1199 Bin'an Road,"hubspot, react redux, remote, route 53, vuejs",NaN,NaN,http://www.dahuasecurity.com,Saudi Arabia,yahya.asfari@dahuatech.com,Yahya,Yahya Asfari,engineering,Director Of Business Development | Ai Surveill...,Information Technology & Services,Director Of Business Development | Ai Surveill...,"dvrs, nvr, ivs, ip cameras, security solution ...",Asfari,http://www.linkedin.com/in/yahya1asfari,NaN,NaN,director,Makkah Province
3,Bogota,32390000000.00,32.4B,Hangzhou Shi,China,"Zhejiang Dahua Technology Co., Ltd. (Dahua Tec...",dahuasecurity.com,2001,"No.1199, Bin'an Road, Binjiang District, Hangz...",http://www.linkedin.com/company/dahua-technology,2845844,Dahua Technology Co. Ltd,+86 571 8768 8883,NaN,23000,Zhejiang Sheng,1199 Bin'an Road,"hubspot, react redux, remote, route 53, vuejs",NaN,NaN,http://www.dahuasecurity.com,Colombia,ruben.trujillo@dahuatech.com,Jose,Jose Ruben Trujillo,sales,Business Development Director,Information Technology & Services,Business Development Director,"dvrs, nvr, ivs, ip cameras, security solution ...",Ruben Trujillo,http://www.linkedin.com/in/jruben-trujillo,NaN,NaN,director,Bogota
4,Beirut,32390000000.00,32.4B,Hangzhou Shi,China,"Zhejiang Dahua Technology Co., Ltd. (Dahua Tec...",dahuasecurity.com,2001,"No.1199, Bin'an Road, Binjiang District, Hangz...",http://www.linkedin.com/company/dahua-technology,2845844,Dahua Technology Co. Ltd,+86 571 8768 8883,NaN,23000,Zhejiang Sheng,1199 Bin'an Road,"hubspot, react redux, remote, route 53, vuejs",NaN,NaN,http://www.dahuasecurity.com,Lebanon,NaN,Rodolphe,Rodolphe Kassouf,sales,Director Of Business Development,Information Technology & Services,Director Of Business Development,"dvrs, nvr, ivs, ip cameras, security solution ...",Kassouf,http://www.linkedin.com/in/rodolphe-kassouf,NaN,

In [8]:
df_dealroom = pd.read_csv(r'C:\Users\LENOVO\Documents\GitHub\Enrichment_database\dataset_dealroom-profile-scraper_large_company_Abridge (1).csv')
df_dealroom.head()

,about,about_ai_generated,analytics/code,analytics/error,analytics/message,appstore_app_id,can_edit,career_page_url,client_focus/0/count,client_focus/0/name,client_focus/1/count,client_focus/1/name,closing_month,closing_year,company_status,current_and_prev_year_fundings_num,delivery_method,employee_12_months_growth_percentile,employee_12_months_growth_unique,employees,employees_chart/0/date,employees_chart/0/value,employees_chart/1/date,employees_chart/1/value,employees_chart/2/date,employees_chart/2/value,employees_chart/3/date,employees_chart/3/value,employees_chart/4/date,employees_chart/4/value,employees_chart/5/date,employees_chart/5/value,employees_chart/6/date,employees_chart/6/value,employees_chart/7/date,employees_chart/7/value,employees_chart/8/date,employees_chart/8/value,employees_chart/9/date,employees_chart/9/value,employees_chart/10/date,employees_chart/10/value,employees_chart/11/date,employees_chart/11/value,employees_chart/12/date,employees_chart/12/value,employees_chart/13/date,employees_chart/13/value,employees_chart/14/date,employees_chart/14/value,employees_chart/15/date,employees_chart/15/value,employees_chart/16/date,employees_chart/16/value,employees_chart/17/date,employees_chart/17/value,employees_chart/18/date,employees_chart/18/value,employees_chart/19/date,employees_chart/19/value,employees_chart/20/date,employees_chart/20/value,employees_chart/21/date,employees_chart/21/value,employees_chart/22/date,employees_chart/22/value,employees_chart/23/date,employees_chart/23/value,employees_chart/24/date,employees_chart/24/value,employees_chart/25/date,employees_chart/25/value,employees_chart/26/date,employees_chart/26/value,employees_chart/27/date,employees_chart/27/value,employees_chart/28/date,employees_chart/28/value,employees_chart/29/date,employees_chart/29/value,employees_chart/30/date,employees_chart/30/value,employees_chart/31/date,employees_chart/31/value,employees_chart/32/date,employees_chart/32/value,employees_chart/33/date,employees_chart/33/value,employees_chart/34/date,employees_chart/34/value,employees_chart/35/date,employees_chart/35/value,employees_chart/36/date,employees_chart/36/value,employees_chart/37/date,employees_chart/37/value,employees_chart/38/date,employees_chart/38/value,employees_chart/39/date,employees_chart/39/value,employees_chart/40/date,employees_chart/40/value,employees_chart/41/date,employees_chart/41/value,employees_chart/42/date,employees_chart/42/value,employees_chart/43/date,employees_chart/43/value,employees_latest,exits_higher_800m,facebook_url,fundings/0/amount,fundings/0/amount_source,fundings/0/currency,fundings/0/id,fundings/0/is_verified,fundings/0/month,fundings/0/news_source,fundings/0/round,fundings/0/standardised_round_label,fundings/0/transaction_multiples/ev_ebitda_multiple,fundings/0/transaction_multiples/ev_ltm_ebitda_multiple,fundings/0/transaction_multiples/ev_ltm_profit_multiple,fundings/0/transaction_multiples/ev_ltm_revenue_multiple,fundings/0/transaction_multiples/ev_ntm_ebitda_multiple,fundings/0/transaction_multiples/ev_ntm_profit_multiple,fundings/0/transaction_multiples/ev_ntm_revenue_multiple,fundings/0/transaction_multiples/ev_profit_multiple,fundings/0/transaction_multiples/ev_revenue_multiple,fundings/0/valuation,fundings/0/valuation_generated_max,fundings/0/valuation_generated_min,fundings/0/year,fundings/1/amount,fundings/1/amount_source,fundings/1/currency,fundings/1/id,fundings/1/is_verified,fundings/1/month,fundings/1/news_source,fundings/1/round,fundings/1/standardised_round_label,fundings/1/transaction_multiples/ev_ebitda_multiple,fundings/1/transaction_multiples/ev_ltm_ebitda_multiple,fundings/1/transaction_multiples/ev_ltm_profit_multiple,fundings/1/transaction_multiples/ev_ltm_revenue_multiple,fundings/1/transaction_multiples/ev_ntm_ebitda_multiple,fundings/1/transaction_multiples/ev_ntm_profit_multiple,fundings/1/transaction_multiples/ev_ntm_revenue_multiple,fundings/1/transaction_multiples/ev_profit_multiple,fundings/1/tran